In [1]:
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc
import scipy.io
from scipy.sparse import csr_matrix
print(ad.__version__)

0.10.9


In [18]:
counts = scipy.io.mmread('data/primary_sparse_processed.mtx').tocsr().transpose()

In [19]:
# Load barcodes and genes
barcodes = pd.read_csv('data/primary_colnames_processed.txt', header=None).squeeze().tolist()
genes = pd.read_csv('data/primary_features_processed.txt', header=None).squeeze().tolist()
print(len(barcodes), len(genes), counts.shape)

156572 25129 (156572, 25129)


In [20]:
adata = sc.AnnData(X=counts)
adata.var_names = genes
adata.obs_names = barcodes


In [31]:
metadata = pd.read_table('data/scp_primary_metadata.txt', index_col=0)
# Remove the first row
metadata = metadata.iloc[1:]
adata.obs = metadata


/var/folders/pj/g7ctw93j7477th9q941xfyyc0000gn/T/ipykernel_87140/3849739391.py:1: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_table('data/scp_primary_metadata.txt', index_col=0)


In [64]:
adata.obs

,biosample_id,donor_id,species,species__ontology_label,sex,disease,disease__ontology_label,organ,organ__ontology_label,organ_custom,library_preparation_protocol,library_preparation_protocol__ontology_label,cell_type,cell_type__ontology_label,cell_type_custom,number_of_reads,number_of_features
NAME,,,,,,,,,,,,,,,,,
D14_OE.AAACCCAGTATTCCTT,D14_OM,D14_Group,NCBITaxon_10090,Mus musculus,female,MONDO_0005812,influenza,UBERON_0001826,nasal cavity mucosa,OM,EFO_0009922,10x 3' v3,CL:0000236,B cell,MT-Hi B,1762,966
D14_OE.AAACGAACAAAGCTCT,D14_OM,D14_Group,NCBITaxon_10090,Mus musculus,female,MONDO_0005812,influenza,UBERON_0001826,nasal cavity mucosa,OM,EFO_0009922,10x 3' v3,CL:0000236,B cell,Nme1/Nme2 Hi B,2175,969
D14_OE.AAACGAAGTGTTAAAG,D14_OM,D14_Group,NCBITaxon_10090,Mus musculus,female,MONDO_0005812,influenza,UBERON_0001826,nasal cavity mucosa,OM,EFO_0009922,10x 3' v3,CL:0000236,B cell,Immature B,893,573
D14_OE.AAACGCTAGAATACAC,D14_OM,D14_Group,NCBITaxon_10090,Mus musculus,female,MONDO_0005812,influenza,UBERON_0001826,nasal cavity mucosa,OM,EFO_0009922,10x 3' v3,CL:0000236,B cell,Immature B,1459,843
D14_OE.AAACGCTAGCATCCTA,D14_OM,D14_Group,NCBITaxon_10090,Mus musculus,female,MONDO_0005812,influenza,UBERON_0001826,nasal cavity mucosa,OM,EFO_0009922,10x 3' v3,CL:0000236,B cell,Mature B 2,1210,590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Naive_RE.CTCATGCAGGCTTAGG,Naive_RM,Naive_Group,NCBITaxon_10090,Mus musculus,female,PATO_0000461,normal,UBERON_0001826,nasal cavity mucosa,RM,EFO_0009922,10x 3' v3,CL:0000037,hematopoietic stem cell,HSC,14808,3745
Naive_RE.GTATTGGGTGCCGGTT,Naive_RM,Naive_Group,NCBITaxon_10090,Mus musculus,female,PATO_0000461,normal,UBERON_0001826,nasal cavity mucosa,RM,EFO_0009922,10x 3' v3,CL:0000037,hematopoietic stem cell,HSC,11051,2989
Naive_RE.TGGGTTATCGCAATGT,Naive_RM,Naive_Group,NCBITaxon_10090,Mus musculus,female,PATO_0000461,normal,UBERON_0001826,nasal cavity mucosa,RM,EFO_0009922,10x 3' v3,CL:0000037,hematopoietic stem cell,HSC,8405,2644


In [58]:
columns_to_convert = ['biosample_id', 'donor_id', 'species', 'species__ontology_label', 'sex',
       'disease', 'disease__ontology_label', 'organ', 'organ__ontology_label',
       'organ_custom', 'library_preparation_protocol',
       'library_preparation_protocol__ontology_label', 'cell_type',
       'cell_type__ontology_label', 'cell_type_custom', 'number_of_reads',
       'number_of_features']
adata.obs[columns_to_convert] = adata.obs[columns_to_convert].astype('category')


In [66]:
adata.write('flu_processed.h5ad', compression="gzip")
